In [3]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('..')
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [4]:
import rnacomp
import rnacomp.models
from rnacomp.fastai_fit import *
from rnacomp.dataset import LenMatchBatchSampler, DeviceDataLoader
from rnacomp.utils import seed_everything, MAE, loss_laplace
import gc
import wandb 
from fastai.callback.wandb import WandbCallback

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [5]:
class CFG:
    path = Path("../data/")
    pathbb = Path("../data/Ribonanza_bpp_files")
    pathss = Path("../eda/train_ss_vienna_rna.parquet")
    split_id = Path('../eda/fold_split.csv')
    bs = 64
    num_workers = 12
    device = 'cuda'
    seed = 2023
    out = 'exp_19'
    dataset_name = 'RNA_DatasetBaselineSplitssbppV1'
    sn_train = False
    
    model_name = 'RNA_ModelV14'
    dim = 192 * 2
    depth = 12
    dim_head = 32
   
    epoch = 64
    lr = 5e-4
    wd = 0.05
    pct_start = 0.02
    
    wandb = True
    
seed_everything(CFG.seed)
os.makedirs(CFG.out, exist_ok=True)

# wandb.init(
#     # set the wandb project where this run will be logged
#     project="my-awesome-project",
    
#     # track hyperparameters and run metadata
#     config={
#     "learning_rate": 0.02,
#     "architecture": "CNN",
#     "dataset": "CIFAR-100",
#     "epochs": 10,
#     }
# )

In [6]:
def class_to_dict(cls):
    # Create a dictionary from the class attributes
    return {key: value for key, value in cls.__dict__.items() if not key.startswith("__") and not callable(value)}

In [7]:
fns = list(CFG.pathbb.rglob("*.txt"))
bpp_df = pd.DataFrame({"bpp": fns})
bpp_df['sequence_id'] = bpp_df['bpp'].apply(lambda x: x.stem)
bpp_df.drop_duplicates(subset = 'sequence_id', inplace=True)
ss = pd.read_parquet(CFG.pathss)[["sequence_id", "ss_full"]]
df = pd.read_parquet(CFG.path/'train_data.parquet')
split = pd.read_csv(CFG.split_id)
df = pd.merge(df, split, on='sequence_id')
df = pd.merge(df, bpp_df, on='sequence_id')
df = pd.merge(df, ss,  on='sequence_id')
#df = df.query("SN_filter==1").reset_index(drop=True)
df_train = df.query('is_train==True').reset_index(drop=True)
df_valid = df.query('is_train==False').reset_index(drop=True)


ds_train = getattr(rnacomp.dataset, CFG.dataset_name)(df_train, mode='train',sn_train=CFG.sn_train)
ds_train_len = getattr(rnacomp.dataset, CFG.dataset_name)(df_train, mode='train', mask_only=True,sn_train=CFG.sn_train)
sampler_train = torch.utils.data.RandomSampler(ds_train_len)
len_sampler_train = LenMatchBatchSampler(sampler_train, batch_size=CFG.bs,
            drop_last=True)
dl_train = DeviceDataLoader(torch.utils.data.DataLoader(ds_train, 
            batch_sampler=len_sampler_train, num_workers=CFG.num_workers,
            persistent_workers=True), CFG.device)


ds_val = getattr(rnacomp.dataset, CFG.dataset_name)(df_valid, mode='eval')
ds_val_len = getattr(rnacomp.dataset, CFG.dataset_name)(df_valid, mode='eval', mask_only=True)
sampler_val = torch.utils.data.SequentialSampler(ds_val_len)
len_sampler_val = LenMatchBatchSampler(sampler_val, batch_size=CFG.bs, 
               drop_last=False)
dl_val= DeviceDataLoader(torch.utils.data.DataLoader(ds_val, 
               batch_sampler=len_sampler_val, num_workers=CFG.num_workers), CFG.device)

data = DataLoaders(dl_train,dl_val)
gc.collect()

16

In [ ]:
if CFG.wandb:
    wandb.init(
        project="rna",
        entity="drhb",
        name=CFG.out,
        config=class_to_dict(CFG),
    )
learn = Learner(data,
                getattr(rnacomp.models, CFG.model_name)(dim = CFG.dim).cuda(), 
                path = CFG.out, 
                loss_func=loss_laplace,
                cbs=[GradientClip(3.0),
                     WandbCallback(log_preds=False),
                    CSVLogger(),
                    SaveModelCallback(monitor='mae',comp=np.less,at_end=True)],
                metrics=[MAE()]).to_fp16() 
learn.fit_one_cycle(CFG.epoch, lr_max=CFG.lr, wd=CFG.wd, pct_start=CFG.pct_start)
wandb.finish()

wandb: Currently logged in as: drhb. Use `wandb login --relogin` to force relogin


Could not gather input dimensions


epoch,train_loss,valid_loss,mae,time
0,0.127235,0.139233,0.149560,29:53
1,0.121241,0.133527,0.144027,29:59
2,0.119641,0.132439,0.141096,30:02
3,0.118421,0.129438,0.137472,30:04
4,0.117907,0.128383,0.136768,30:00
5,0.116862,0.128328,0.136833,30:02
6,0.117164,0.127424,0.136106,30:08
7,0.116527,0.125316,0.134996,30:02
8,0.115069,0.124729,0.134390,30:03
9,0.115670,0.126270,0.135039,30:04


Better model found at epoch 0 with mae value: 0.14956006114459686.


IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



...

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



Better model found at epoch 30 with mae value: 0.12999202157373213.
Better model found at epoch 31 with mae value: 0.12978196929730815.
Better model found at epoch 32 with mae value: 0.12973371949054885.
Better model found at epoch 33 with mae value: 0.12968467779251736.


IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



Better model found at epoch 38 with mae value: 0.12918220355560206.
Better model found at epoch 40 with mae value: 0.1288348293996265.


IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [ ]:
res = dict()
for l in df_valid["L"].unique():
    ds_val = getattr(rnacomp.dataset, CFG.dataset_name)(df_valid.query("L==@l").copy(), mode='eval')
    ds_val_len = getattr(rnacomp.dataset, CFG.dataset_name)(df_valid.query("L==@l").copy(), mode='eval', mask_only=True)
    sampler_val = torch.utils.data.SequentialSampler(ds_val_len)
    len_sampler_val = LenMatchBatchSampler(sampler_val, batch_size=CFG.bs, 
                   drop_last=False)
    dl_val= DeviceDataLoader(torch.utils.data.DataLoader(ds_val, 
                   batch_sampler=len_sampler_val, num_workers=CFG.num_workers), CFG.device)

    data = DataLoaders(dl_val,dl_val)
    learn = Learner(data,
                    getattr(rnacomp.models, CFG.model_name)().cuda(), 
                    path = CFG.out, 
                    loss_func=loss_laplace,
                    metrics=[MAE()]).to_fp16() 
    learn.load('model')
    learn.eval()
    loss_, score_ = learn.validate()
    res[l]= score_

res = pd.DataFrame(pd.Series(res)).reset_index()
res.columns = ['L', 'mae']
res.sort_values(by="L")

In [ ]:
res

In [ ]:
res